In [1]:
!pip install dotenv
from dotenv import load_dotenv
import os

  Using cached dotenv-0.9.9-py2.py3-none-any.whl (1.9 kB)

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
# Fetch API keys securely
DATAMALL_API_KEY = os.getenv("DATAMALL_API_KEY")
ONEMAP_API_KEY = os.getenv("ONEMAP_API_KEY")

In [ ]:
%pip install requests
import requests

url = "https://datamall2.mytransport.sg/ltaodataservice/PV/Train"
headers = {
    "AccountKey": DATAMALL_API_KEY ,
    "accept": "application/json"
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    print(response.json())  # Returns MRT station data
else:
    print("Error:", response.status_code)


  Using cached requests-2.32.3-py3-none-any.whl (64 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.4/128.4 kB 4.2 MB/s eta 0:00:00
  Using cached certifi-2025.1.31-py3-none-any.whl (166 kB)
  Using cached charset_normalizer-3.4.1-cp39-cp39-macosx_10_9_universal2.whl (197 kB)

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
{'odata.metadata': 'http://datamall2.mytransport.sg/ltaodataservice/$metadata#FarecardBatch', 'value': [{'Link': 'https://ltafarecard.s3.ap-southeast-1.amazonaws.com/202502/transport_node_train_202502.zip?X-Amz-Security-Token=IQoJb3JpZ2luX2VjEC4aDmFwLXNvdXRoZWFzdC0xIkgwRgIhAKgmK99KTMoSz6CU9krvzPEs3BOZXisHRpfcEGqQyJe4AiEArv85uZDlP7WJz9itlAp1oSl583doVvwiT2sRLa3xLckqywUIh%2F%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FARAEGgwzNDA2NDUzODEzMDQiDLDDjg39TyX0nFKqbCqfBbHKxjLLOUtFaAxeKSfbTVTsqK17B5Apjj%2FlCKz98QE5MyxqvFCQdCooSyxjCuOm13RHnqugAjQo9gIDpRxSYPrhL

In [73]:
import requests
import zipfile
import io

url = "https://datamall2.mytransport.sg/ltaodataservice/PV/Train"
headers = {
    "AccountKey": "mTqjelL7QuuYXyLysHP+ug==",
    "accept": "application/json"
}

# Request data from the API
response = requests.get(url, headers=headers)
response.raise_for_status()  # Ensure there are no errors

# Extract the download link
download_url = response.json()["value"][0]["Link"]

# Download the ZIP file
zip_response = requests.get(download_url)
zip_response.raise_for_status()

# Extract the ZIP file in memory
with zipfile.ZipFile(io.BytesIO(zip_response.content), "r") as zip_ref:
    zip_ref.extractall("train_data")  # Extracts files into a folder




In [74]:
import json
import pandas as pd
import re

with open("LTAMRTStation.geojson", "r", encoding="utf-8") as f:
    geojson_data = json.load(f)


# Extract relevant information
data = []
for feature in geojson_data["features"]:
    properties = feature["properties"]
    geometry = feature["geometry"]
    
    # Extract HTML description and parse attributes
    description = properties["Description"]
    
    # Use regex to extract values
    station_match = re.search(r"<th>STATION_NA<\/th> <td>(.*?)<\/td>", description)
    exit_match = re.search(r"<th>EXIT_CODE<\/th> <td>(.*?)<\/td>", description)

    station_name = station_match.group(1) if station_match else "Unknown"
    exit_code = exit_match.group(1) if exit_match else "Unknown"
    
    # Extract coordinates
    longitude, latitude, _ = geometry["coordinates"]
    
    # Append to list
    data.append([station_name, exit_code, latitude, longitude])

df = pd.DataFrame(data, columns=["Station Name", "Exit Code", "Latitude", "Longitude"])
df.head()
df = df.sort_values('Station Name')
df.to_csv("mrt_stations_exits.csv", index=False)


In [75]:
import requests

# Get all MRT Stations Address in Singapore.
full_MRT_data = []

#EWL Stations
EWL_code_list = [f"EW{i}" for i in range(1, 34)]
EWL_code_list.append("CG1")
EWL_code_list.append("CG2")

# North East Line Stations
NEL_code_list = [f"NE{i}" for i in range(1, 19)]
NEL_code_list.remove("NE2")

# North-South Line Stations
NSL_code_list = [f"NS{i}" for i in range(1, 29)]
NSL_code_list.remove("NS6")

# Circle Line Stations
CCL_code_list = [f"CC{i}" for i in range(1, 30)]
CCL_code_list.append("CE1")
CCL_code_list.append("CE2")
CCL_code_list.remove("CC18")

# Thomson-EastCoast Line Stations
TEL_code_list = [f"TE{i}" for i in range(1, 32)]
TEL_code_list.remove("TE10")
TEL_code_list.remove("TE21")

# Downtown Line Stations
DTL_code_list = [f"DT{i}" for i in range(1, 38)]

full_mrt_code = EWL_code_list + NEL_code_list + NSL_code_list + CCL_code_list + TEL_code_list + DTL_code_list

full_MRT_data = []

for codeName in full_mrt_code:
    url = f"https://www.onemap.gov.sg/api/common/elastic/search?searchVal={codeName}&returnGeom=Y&getAddrDetails=Y&pageNum=1"
    response = requests.get(url)
    data = response.json()

    if "results" in data and isinstance(data["results"], list):
        for result in data["results"]:
            if "SEARCHVAL" in result and "MRT" in result["SEARCHVAL"]:
                full_MRT_data.append(result)
                break  # Stop after finding the first correct match

# Print or store full MRT station data
print(json.dumps(full_MRT_data, indent=4))




[
    {
        "SEARCHVAL": "PASIR RIS MRT STATION (EW1)",
        "BLK_NO": "10",
        "ROAD_NAME": "PASIR RIS CENTRAL",
        "BUILDING": "PASIR RIS MRT STATION (EW1)",
        "ADDRESS": "10 PASIR RIS CENTRAL PASIR RIS MRT STATION (EW1) SINGAPORE 519634",
        "POSTAL": "519634",
        "X": "40905.597462655",
        "Y": "39449.9823396821",
        "LATITUDE": "1.37304331635804",
        "LONGITUDE": "103.949284527763"
    },
    {
        "SEARCHVAL": "TAMPINES MRT STATION (EW2)",
        "BLK_NO": "20",
        "ROAD_NAME": "TAMPINES CENTRAL 1",
        "BUILDING": "TAMPINES MRT STATION (EW2)",
        "ADDRESS": "20 TAMPINES CENTRAL 1 TAMPINES MRT STATION (EW2) SINGAPORE 529538",
        "POSTAL": "529538",
        "X": "40444.9732025806",
        "Y": "37267.0262664594",
        "LATITUDE": "1.35330167637376",
        "LONGITUDE": "103.945144586749"
    },
    {
        "SEARCHVAL": "SIMEI MRT STATION (EW3)",
        "BLK_NO": "30",
        "ROAD_NAME": "SIMEI STREET

In [76]:
import requests
import json

url = "https://www.onemap.gov.sg/api/common/elastic/search?searchVal=NE3&returnGeom=Y&getAddrDetails=Y&pageNum=1"

response = requests.get(url)
data = response.json()  # Convert response text to JSON

# Pretty print JSON response
print(json.dumps(data, indent=4))


{
    "found": 1,
    "totalNumPages": 1,
    "pageNum": 1,
    "results": [
        {
            "SEARCHVAL": "OUTRAM PARK MRT STATION (NE3)",
            "BLK_NO": "300",
            "ROAD_NAME": "EU TONG SEN STREET",
            "BUILDING": "OUTRAM PARK MRT STATION (NE3)",
            "ADDRESS": "300 EU TONG SEN STREET OUTRAM PARK MRT STATION (NE3) SINGAPORE 059816",
            "POSTAL": "059816",
            "X": "28681.2492930563",
            "Y": "29130.5222722683",
            "LATITUDE": "1.27972059383553",
            "LONGITUDE": "103.839439862425"
        }
    ]
}


In [78]:
df = pd.DataFrame(full_MRT_data)
df = df.drop_duplicates()

# Save to CSV
csv_filename = "all_mrt_stations.csv"
df.to_csv(csv_filename, index=False)

In [79]:
df_filtered = df.drop(columns=['SEARCHVAL', 'BLK_NO', 'X', 'Y'])

# Save to a new CSV file
csv_filtered_filename = "mrt_stations_filtered.csv"
df_filtered.to_csv(csv_filtered_filename, index=False)


In [24]:
import requests
import json
import pandas as pd

# LRT Stations
full_LRT_data = []

# SBS LRT
punggol_lrt_codes = ["Punggol LRT Station"]
# Punggol East (PE)
PE_code_list = [f"PE{i}" for i in range(1, 8)]

# Punggol West (PW)
PW_code_list = [f"PW{i}" for i in range(1, 8)]

punggol_lrt_codes += PE_code_list + PW_code_list
print(punggol_lrt_codes)

sengkang_lrt_codes = ["STC"]
# Sengkang East (SE)
SE_code_list = [f"SE{i}" for i in range(1, 6)]

# Sengkang West (SW)
SW_code_list = [f"SW{i}" for i in range(1, 9)]

sengkang_lrt_codes += SE_code_list + SW_code_list
print(sengkang_lrt_codes)

# SMRT LRT
smrt_lrt_codes = [f"BP{i}" for i in range(1, 14)]
print(smrt_lrt_codes)

full_lrt_code = []
full_lrt_code += punggol_lrt_codes + sengkang_lrt_codes + smrt_lrt_codes
print(full_lrt_code)

for codeName in full_lrt_code:
    url = f"https://www.onemap.gov.sg/api/common/elastic/search?searchVal={codeName}&returnGeom=Y&getAddrDetails=Y&pageNum=1"
    response = requests.get(url)
    data = response.json()

    if "results" in data and isinstance(data["results"], list):
        for result in data["results"]:
            if "SEARCHVAL" in result and "LRT" in result["SEARCHVAL"]:
                full_LRT_data.append(result)
                break  # Stop after finding the first correct match

# Print or store full MRT station data
print(json.dumps(full_LRT_data, indent=4))

df = pd.DataFrame(full_LRT_data)
df = df.drop_duplicates()

# Save to CSV
csv_filename = "all_lrt_stations.csv"
df.to_csv(csv_filename, index=False)

df_filtered = df.drop(columns=['SEARCHVAL', 'BLK_NO', 'X', 'Y'])

# Save to a new CSV file
csv_filtered_filename = "lrt_stations_filtered.csv"
df_filtered.to_csv(csv_filtered_filename, index=False)



['Punggol LRT Station', 'PE1', 'PE2', 'PE3', 'PE4', 'PE5', 'PE6', 'PE7', 'PW1', 'PW2', 'PW3', 'PW4', 'PW5', 'PW6', 'PW7']
['STC', 'SE1', 'SE2', 'SE3', 'SE4', 'SE5', 'SW1', 'SW2', 'SW3', 'SW4', 'SW5', 'SW6', 'SW7', 'SW8']
['BP1', 'BP2', 'BP3', 'BP4', 'BP5', 'BP6', 'BP7', 'BP8', 'BP9', 'BP10', 'BP11', 'BP12', 'BP13']
['Punggol LRT Station', 'PE1', 'PE2', 'PE3', 'PE4', 'PE5', 'PE6', 'PE7', 'PW1', 'PW2', 'PW3', 'PW4', 'PW5', 'PW6', 'PW7', 'STC', 'SE1', 'SE2', 'SE3', 'SE4', 'SE5', 'SW1', 'SW2', 'SW3', 'SW4', 'SW5', 'SW6', 'SW7', 'SW8', 'BP1', 'BP2', 'BP3', 'BP4', 'BP5', 'BP6', 'BP7', 'BP8', 'BP9', 'BP10', 'BP11', 'BP12', 'BP13']
[
    {
        "SEARCHVAL": "PUNGGOL LRT STATION (PTC)",
        "BLK_NO": "70",
        "ROAD_NAME": "PUNGGOL CENTRAL",
        "BUILDING": "PUNGGOL LRT STATION (PTC)",
        "ADDRESS": "70 PUNGGOL CENTRAL PUNGGOL LRT STATION (PTC) SINGAPORE 828868",
        "POSTAL": "828868",
        "X": "35689.1389922583",
        "Y": "43004.9202214088",
        "LATITUDE":

In [19]:

import requests
import json

url = "https://www.onemap.gov.sg/api/common/elastic/search?searchVal=Punggol LRT Station&returnGeom=Y&getAddrDetails=Y&pageNum=1"

response = requests.get(url)
data = response.json()  # Convert response text to JSON

# Pretty print JSON response
print(json.dumps(data, indent=4))

{
    "found": 1,
    "totalNumPages": 1,
    "pageNum": 1,
    "results": [
        {
            "SEARCHVAL": "PUNGGOL LRT STATION (PTC)",
            "BLK_NO": "70",
            "ROAD_NAME": "PUNGGOL CENTRAL",
            "BUILDING": "PUNGGOL LRT STATION (PTC)",
            "ADDRESS": "70 PUNGGOL CENTRAL PUNGGOL LRT STATION (PTC) SINGAPORE 828868",
            "POSTAL": "828868",
            "X": "35689.1389922583",
            "Y": "43004.9202214088",
            "LATITUDE": "1.40519470149606",
            "LONGITUDE": "103.902411911915"
        }
    ]
}


In [13]:
import requests
import pandas as pd

# Get all bus routes in Singapore.

API_KEY = "mTqjelL7QuuYXyLysHP+ug=="
URL = "http://datamall2.mytransport.sg/ltaodataservice/BusRoutes"

bus_routes = []
skip = 0  # Start at the first page

while True:
    response = requests.get(f"{URL}?$skip={skip}", headers=headers)
    data = response.json().get("value", [])

    if not data: 
        break

    bus_routes.extend(data)
    skip += 500 

print(f"Total bus routes fetched: {len(bus_routes)}")

# Convert to DataFrame
df = pd.DataFrame(bus_routes)

# Sort DataFrame by 'ServiceNo'
df_sorted = df.sort_values(by="ServiceNo")
df_sorted.head()



Total bus routes fetched: 25899


,ServiceNo,Operator,Direction,StopSequence,BusStopCode,Distance,WD_FirstBus,WD_LastBus,SAT_FirstBus,SAT_LastBus,SUN_FirstBus,SUN_LastBus
0,10,SBST,1,1,75009,0.0,0500,2300,0500,2300,0500,2300
95,10,SBST,2,22,14139,6.9,0621,0017,0620,0019,0634,0017
96,10,SBST,2,23,14129,7.4,0622,0018,0621,0020,0635,0018
97,10,SBST,2,24,14119,7.7,0623,0019,0622,0021,0636,0019
98,10,SBST,2,25,14109,8.4,0624,0020,0623,0022,0637,0020


In [16]:
URL = "https://datamall2.mytransport.sg/ltaodataservice/BusStops"
API_KEY = "mTqjelL7QuuYXyLysHP+ug=="


bus_stops = []
skip = 0  # Start at the first page

while True:
    response = requests.get(f"{URL}?$skip={skip}", headers=headers)
    data = response.json().get("value", [])

    if not data: 
        break

    bus_stops.extend(data)
    skip += 500 

print(f"Total bus routes fetched: {len(bus_stops)}")

df = pd.DataFrame(bus_stops)
df.to_csv("bus_stops.csv", index="False")

Total bus routes fetched: 5153


In [25]:
# Inner Join bus_routes.csv with bus_stops.csv ON Bus Stop Code

bus_stops_df = pd.read_csv("bus_stops.csv")   # Contains BusStopCode and bus stop details
bus_routes_df = pd.read_csv("bus_routes.csv")  # Contains BusStopCode and bus route details

# Ensure BusStopCode is treated as a string (in case of leading zeros)
bus_stops_df["BusStopCode"] = bus_stops_df["BusStopCode"].astype(str)
bus_routes_df["BusStopCode"] = bus_routes_df["BusStopCode"].astype(str)

# Perform the join on BusStopCode
merged_df = bus_routes_df.merge(bus_stops_df, on="BusStopCode", how="inner") 
merged_df.head()


,Unnamed: 0_x,ServiceNo,Operator,Direction,StopSequence,BusStopCode,Distance,WD_FirstBus,WD_LastBus,SAT_FirstBus,SAT_LastBus,SUN_FirstBus,SUN_LastBus,Unnamed: 0_y,RoadName,Description,Latitude,Longitude
0,0,10,SBST,1,1,75009,0.0,0500,2300,0500,2300,0500,2300,4183,Tampines Ctrl 1,Tampines Int,1.354076,103.943391
1,1,10,SBST,1,2,76059,0.6,0502,2302,0502,2302,0502,2302,4287,Tampines Ave 5,Opp Our Tampines Hub,1.352962,103.941652
2,2,10,SBST,1,3,76069,1.1,0504,2304,0504,2304,0503,2304,4289,Tampines Ave 5,Blk 147,1.348753,103.942086
3,3,10,SBST,1,4,96289,2.3,0508,2308,0508,2309,0507,2308,5021,Simei Ave,Changi General Hosp,1.340055,103.948381
4,4,10,SBST,1,5,96109,2.7,0509,2310,0509,2311,0508,2309,4987,Simei Ave,Opp Blk 3012,1.337371,103.950673


In [46]:
merged_df = pd.DataFrame(merged_df, columns=["ServiceNo", "Operator", "Direction", "BusStopCode", "RoadName", "Description", "Latitude", "Longitude"])
merged_df.head()
merged_df.to_csv("bus_routes_detailed.csv", index="False")

In [66]:

bus_routes_df = pd.read_csv("bus_routes_detailed.csv")

# Count the number of unique bus services passing by each bus stop
bus_stop_service_count = bus_routes_df.groupby("BusStopCode")["ServiceNo"].nunique().reset_index()
bus_stop_service_count.rename(columns={"ServiceNo": "NumBusServices"}, inplace=True)
bus_stop_details = bus_routes_df.drop_duplicates(subset=["BusStopCode"])[
    ["BusStopCode", "RoadName", "Description", "Latitude", "Longitude"]
]

bus_stop_summary = bus_stop_details.merge(bus_stop_service_count, on="BusStopCode", how="left")
bus_stop_summary["BusStopCode"] = bus_stop_summary["BusStopCode"].astype(str)
bus_stop_summary_sorted = bus_stop_summary.sort_values(by="BusStopCode")
bus_stop_summary_sorted.to_csv("Bus_Stop_To_NumberOfBusServices.csv", index=False)


In [ ]:
# Get the list of buses in each stop.
bus_stop_summary_sorted["BusStopCode"] = bus_stop_summary_sorted["BusStopCode"].astype(str)
bus_routes_df["BusStopCode"] = bus_routes_df["BusStopCode"].astype(str)

bus_stop_services = bus_routes_df.groupby("BusStopCode")["ServiceNo"].apply(list).reset_index()
bus_stop_services.rename(columns={"ServiceNo": "BusServicesList"}, inplace=True)

bus_stop_summary_with_services = bus_stop_summary_sorted.merge(bus_stop_services, on="BusStopCode", how="left")

bus_stop_summary_with_services.to_csv("Bus_Stop_With_NumOfBus_And_ServicesList.csv", index=False)

bus_stop_services_cleaned = bus_routes_df.groupby("BusStopCode")["ServiceNo"].apply(lambda x: list(set(x))).reset_index()
bus_stop_services_cleaned.rename(columns={"ServiceNo": "BusServicesList"}, inplace=True)

bus_stop_services_cleaned["NumBusServices"] = bus_stop_services_cleaned["BusServicesList"].apply(len)

bus_stop_summary_cleaned = bus_stop_summary_sorted.drop(columns=["NumBusServices"]).merge(
    bus_stop_services_cleaned, on="BusStopCode", how="left"
)

# Save the corrected dataset
bus_stop_summary_cleaned.to_csv("Bus_Stop_With_ServiceList_Final.csv", index=False)

